# JSON to CSV parser for weather data from online API
After finish running this notebook, a CSV file with hourly weather data for New Yor City in 2016 is created "new_year_city_hourly_weather_2016.csv"

In [ ]:
import pandas as pd
import pytz
import time
from datetime import datetime
import urllib
import json

# get row for data frame


def getDataRow(json):
    nytimestamp = json.get('valid_time_gmt')
    nydatetime = datetime.fromtimestamp(
        nytimestamp, pytz.timezone('US/Eastern'))
    # print(datetime.strftime("%M"))
    if (nydatetime.strftime("%M") != '51'):
        return None
    time = json.get('valid_time_gmt')
    temp = json.get('temp')
    dewp = json.get('dewPt')
    humid = json.get('rh')
    wind = json.get('wspd')
    cond = json.get('wx_phrase')
    data = {'date': nydatetime.strftime("%Y-%m-%d"), 'time': nydatetime.strftime("%H:%M"), 'temperature': temp,
            'dewpoint': dewp, 'humidity': humid, 'windspeed': wind, 'condition': cond}
    return data


url_template = "https://api.weather.com/v1/location/KLGA:9:US/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&units=e&startDate={0}&endDate={0}"

report = pd.DataFrame(columns=['date', 'amount'])
year_df = pd.DataFrame(columns=[
                       'date', 'time', 'temperature', 'dewpoint', 'humidity', 'windspeed', 'condition'])

for month in range(1, 13):
    endmonthday = 31
    if (month == 2):
        endmonthday = 30
    elif (month in [1, 3, 5, 7, 8, 10, 12]):
        endmonthday = 32

    for day in range(1, endmonthday):
        date = "2016{0:02d}{1:02d}".format(month, day)
        url = url_template.format(date)
        time.sleep(1)
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        print(date + ': ' + str(len(data["observations"])))
        for i in data["observations"]:
            data = getDataRow(i)
            if (data != None):
                year_df = year_df.append(data, ignore_index=True)

year_df = year_df.sort_values(by=['date', 'TimeEST'])
year_df.to_csv('new_year_city_hourly_weather_2016.csv', index=False)

print(len(year_df))
print("-------- Finish --------")